In [1]:
from ekonlpy.tag import Mecab
from konlpy.tag import Okt
import pandas as pd
import os

# Mecab & Okt 객체 생성
mecab = Mecab()
okt = Okt()

# 문장에서 한 글자 단어를 문맥에 맞게 변형하는 함수
def adjust_single_char_tokens(sentence):
    pos_result = mecab.pos(sentence)  # 품사 태깅
    adjusted_sentence = []

    for word, pos in pos_result:
        # 한 글자 단어에 대해 문맥을 고려하여 변형
        if len(word) == 1:
            if pos.startswith('V'):  # 동사인 경우
                adjusted_sentence.append(word + "다")  # "있" => "있다"
            elif pos.startswith('A'):  # 형용사인 경우
                adjusted_sentence.append(word + "다")  # "좋" => "좋다"
            else:
                adjusted_sentence.append(word)  # 그 외의 경우는 그대로
        else:
            adjusted_sentence.append(word)

    return " ".join(adjusted_sentence)

# 어간 추출 후 원형 복원하는 함수
def extract_clean_tokens_from_pos(pos_tagged):
    clean_tokens = []
    
    for word, pos in pos_tagged:
        if pos.startswith(('N', 'V', 'A', 'M')):  # 명사, 동사, 형용사, 부사만 선택
            lemma = okt.morphs(word)  # 원형 복원 적용
            clean_tokens.extend(lemma)
    
    # 한 글자 단어를 문맥에 맞게 변형
    adjusted_keywords = [adjust_single_char_tokens(word) for word in clean_tokens]
    
    return adjusted_keywords

# CSV 파일 읽기
df = pd.read_csv(r"C:/Users/wosle/OneDrive/Desktop/프로젝트 파일/뉴스 클랜징/머니투데이/머니투데이전처리T.csv", encoding="utf-8-sig")

# 저장할 경로 설정
output_base_folder = r"C:/Users/wosle/OneDrive/Desktop/프로젝트 파일/뉴스 클랜징/머니투데이"

# 년도별로 데이터 분리하여 저장
for year in df['date'].apply(lambda x: pd.to_datetime(x).year).unique():
    year_df = df[df['date'].apply(lambda x: pd.to_datetime(x).year == year)]
    
    # 월별로 데이터 나누어 저장
    for month in range(1, 13):
        month_df = year_df[year_df['date'].apply(lambda x: pd.to_datetime(x).month == month)]
        
        if month_df.empty:
            continue  # 데이터가 없으면 건너뜀
        
        # 폴더 생성
        folder_name = f"{year}-{str(month).zfill(2)}"
        output_folder = os.path.join(output_base_folder, folder_name)
        os.makedirs(output_folder, exist_ok=True)
        
        # 품사 태깅 적용 (ekonlpy 사용)
        month_df["pos_tagged"] = month_df["orgn_sntc"].apply(lambda x: mecab.pos(str(x)))
        
        # 클린 토큰만 추출 (명사, 동사, 형용사, 부사 포함)
        month_df["clean_token"] = month_df["pos_tagged"].apply(extract_clean_tokens_from_pos)
        
        # 첫 번째 날짜와 마지막 날짜 추출 (파일명에 반영)
        first_date = month_df['date'].min()
        last_date = month_df['date'].max()
        
        # 새로운 파일명 생성
        new_filename = f"token_money_today_{first_date[:4]}{first_date[5:7]}.csv"
        output_path = os.path.join(output_folder, new_filename)
        
        # pos_tagged 열 제거 후 저장
        month_df.drop(columns=['pos_tagged'], inplace=True)
        
        # 파일 저장
        month_df.to_csv(output_path, index=False, encoding="utf-8-sig", quoting=1)
        print(f"{new_filename} 저장 완료.")


token_money_today_202502.csv 저장 완료.
